# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [5]:
from __future__ import print_function 
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
from scipy.stats import norm
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import random

In [6]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [62]:
# number of callbacks for black-sounding names
b_call=sum(data[data.race=='b'].call)
print(b_call)

157.0


In [38]:
# number of callbacks for white-sounding names
w_call= sum(data[data.race=='w'].call)

In [12]:
# total number of callbacks
sum(data.call)

392.0

In [13]:
data.call.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: call, dtype: float32

### Question 1. What test is appropriate for this problem? Does CLT apply?

In this situation we have two outcomes for each resume, call or no call. 
In this case, each resume would be a Bernoulli random variable, and all resume for each name 
has a binomial distribution. The experiment is to test if two binomial distributions are statistically
different from each other. 

The CLT could apply here. The size of the experiment is large, and the call back for each resume is independent from other resumens. However, we have discrete random variable here, and CLT could be used for normal approximation of binomial distribution. 

### Question 2. What are the null and alternate hypotheses?
The null hypthesis is that the names doesn't make a difference in employers' calls, so the probability of getting callback for white or black sounding names are the same.
The alternate hypothesis is white sounding names receive more calls from employers.

### Question 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [27]:
w = data[data.race=='w']
b = data[data.race=='b']
prob_callback=sum(data.call)/len(data)
prob_callback_b=sum(b.call)/len(b)
prob_callback_w=sum(w.call)/len(w)


In [8]:
# bootstrap method

def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)

    return bs_replicates

In [67]:
data_call=data.call

# bootstrap
bs=draw_bs_reps(data_call, sum,10000)/2


# confidence interval
conf_int_bs = np.percentile(bs,[2.5,97.5])
print('95% confidence interval is', conf_int_bs)

# p vale
p=np.sum(bs<=(b_call))/len(bs)
print('p-value =', p)

95% confidence interval is [177.5 215. ]
p-value = 0.0001


In [46]:
# frequentist
# draw sample from a binomial distribution with the probability of success in the whole sample
fr=np.random.binomial(len(b), prob_callback, size=10000)

# confidence interval
conf_int_fr = np.percentile(fr,[2.5,97.5])
print('95% confidence interval is', conf_int_fr)

# p vale
p=np.sum(fr<=b_call)/len(fr)
print('p-value =', p)

95% confidence interval is [170. 223.]
p-value = 0.0016


### Question 4. Write a story describing the statistical significance in the context or the original problem.

The analysis revealed that if we assume there is no discrimination, and that black sounding names would be getting the same amount of callbacks as the general population. However, given the low p-value,the null hypothesis is rejected. The alternate hypothesis suggests that black sounding names are receiving lower call back amounts, suggesting that there is a difference between white and black sounding names. 

### Question 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis shows that race/name is one factore in callback success, but it may not be the most important factor. I would calculate the difference between the callbacks of black and white sounding names, and compare it to the differneces between others factors, and find out which would give the most difference. For example, I would compare race with education. 